In [ ]:
import pandas as pd
import re
import plotly.express as px
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Hi Hanna, you can change this by downloading it from Deepnote :), Good Luck!
final_pred = pd.read_csv('final_pred2.csv')

In [ ]:
final_pred['Crime type'] =final_pred['Crime type'].replace({'Crime type_Anti-social behaviour':'Anti-social behaviour' , 'Crime type_Bicycle theft':'Bicycle theft',
       'Crime type_Burglary':'Burglary', 'Crime type_Criminal damage and arson':'Criminal damage and arson',
       'Crime type_Drugs':'Drugs', 'Crime type_Other crime': 'Other crime',
       'Crime type_Other theft': 'Other theft', 'Crime type_Possession of weapons':'Possession of weapons',
       'Crime type_Public order':'Public order', 'Crime type_Robbery':'Robbery',
       'Crime type_Shoplifting':'Shoplifting', 'Crime type_Theft from the person':'Theft from the person',
       'Crime type_Vehicle crime': 'Vehicle crime',
       'Crime type_Violence and sexual offences':'Violence and sexual offences', 'Sum crimes': 'Total crime in MSOA'})

In [ ]:
df = final_pred[final_pred['prediction'] < 0]
df['prediction'] = 0
df2 = final_pred[final_pred['prediction'] >= 0]
final_pred = pd.concat([df, df2])
final_pred = final_pred.sort_values(by='Month').reset_index(drop=True)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
final_pred.head()

,Unnamed: 0,Month,prediction,min,max,Crime type,MSOA code,change,change_min,change_max,District
0,1716,2021-11-01,9.705307,5.839933,13.532222,Public order,W02000343,119.107803,727.501329,66.301998,Gwent
1,4644,2021-11-01,9.348046,5.668682,13.283612,Vehicle crime,W02000359,35.526300,75.088509,21.648377,Gwent
2,4356,2021-11-01,0.000000,-2.752052,2.616843,Criminal damage and arson,W02000358,-128.443539,20.374978,-17.495585,Gwent
3,4632,2021-11-01,3.056183,1.920921,4.268986,Theft from the person,W02000359,52.285024,105.797110,34.777673,Gwent
4,1764,2021-11-01,3.051105,0.798548,5.099906,Vehicle crime,W02000343,8.014425,9.377612,-1.079605,Gwent


In [ ]:
import plotly.graph_objs as go
# inputs:
msoa = 'W02000343'
crime_type = 'Vehicle crime'

final_pred1 = final_pred[(final_pred['MSOA code'] == msoa) & (final_pred['Crime type'] == crime_type)]
x = final_pred1['Month'].tolist()
y = final_pred1['prediction'].tolist()
y_upper = final_pred1['max'].tolist()
y_lower = final_pred1['min'].tolist()


fig = go.Figure([
    go.Scatter(
        x=x,
        y=y,
        line=dict(color='rgb(0,100,80)'),
        mode='lines'
    ),
    go.Scatter(
        x=x+x[::-1], # x, then x reversed
        y=y_upper+y_lower[::-1], # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    )
])
fig.show()

## Choropleth map predictions per MSOA

In [ ]:
import json

In [ ]:
# this is already filtered in the app
district = 'Gwent'
df = final_pred[final_pred['District'] == district]
# so df should be the data frame with predictions filtered for the district we want


##### copy to app:
# load the geojson
uk_msoas = json.load(open("msoa.geojson", "r"))
#####

In [ ]:
##### copy to app:
# filter the geojson to onlly contain info for our district
uk_msoas_filtered = {"type": "FeatureCollection",
                     "name": "Middle_Layer_Super_Output_Areas__December_2011__Boundaries_Full_Clipped__BFC__EW_V3",
                     "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
                     'features' : []}

for x in range(len(uk_msoas['features'])):
    if uk_msoas['features'][x]['properties']['MSOA11CD'] in [y for y in df['MSOA code'].unique()]:
        uk_msoas_filtered['features'].append({"type": uk_msoas['features'][x]['type'],
                                              "properties" : uk_msoas['features'][x]['properties'],
                                             "geometry" : uk_msoas['features'][x]['geometry']})

#####

In [ ]:
# this will be input from the user - so don't copy this
month = '2022-01-01'
crime_type = 'Drugs'
dff = df[(df['Month'] == month) & (df['Crime type'] ==  crime_type)]
dff.head()

,Unnamed: 0,Month,prediction,min,max,Crime type,MSOA code,change,change_min,change_max,District
1027,410,2022-01-01,1.131228,-0.337488,2.706381,Drugs,W02000336,-35.005480,-405.056185,-18.347078,Gwent
1047,2030,2022-01-01,1.145850,0.063863,2.218748,Drugs,W02000345,-39.701349,-92.192309,-23.726128,Gwent
1057,2390,2022-01-01,1.113798,-0.404490,2.586593,Drugs,W02000347,-45.711816,-162.614085,-27.208710,Gwent
1072,4190,2022-01-01,0.862715,-1.488005,3.438821,Drugs,W02000357,-62.959818,3810.188357,-27.385081,Gwent
1083,230,2022-01-01,0.564278,-0.449858,1.700089,Drugs,W02000335,-15.068481,-9.802537,-3.554351,Gwent


In [ ]:
fig = px.choropleth(dff, geojson=uk_msoas_filtered, color="change",
                    locations="MSOA code", featureidkey="properties.MSOA11CD",
                    color_continuous_scale=px.colors.diverging.RdBu, range_color=[-400, 400])
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, coloraxis_colorbar=dict(title="Change (%)"))

fig.update_geos(fitbounds="locations", visible=False)
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=40789b9a-1c62-45b9-9d9c-b1a39ebe3dfd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>